In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords,movie_reviews
from collections import Counter
import nltk

In [2]:
from nltk.corpus import shakespeare

In [3]:
print(shakespeare.fileids())

['a_and_c.xml', 'dream.xml', 'hamlet.xml', 'j_caesar.xml', 'macbeth.xml', 'merchant.xml', 'othello.xml', 'r_and_j.xml']


In [4]:
print(shakespeare.raw('hamlet.xml'))

<?xml version="1.0"?>
<?xml-stylesheet type="text/css" href="shakes.css"?>
<!-- <!DOCTYPE PLAY SYSTEM "play.dtd"> -->

<PLAY>
<TITLE>The Tragedy of Hamlet, Prince of Denmark</TITLE>

<!--

<P>Text placed in the public domain by Moby Lexical Tools, 1992.</P>
<P>XML markup by Jon Bosak, 1996-1998.</P>
<P>XML Styling done by Ajay Juneja, 1999.<P>
<P>This work may be freely copied and distributed worldwide.</P>

-->

<PERSONAE>
<TITLE>Dramatis Personae</TITLE>

<PERSONA>CLAUDIUS, king of Denmark. </PERSONA>
<PERSONA>HAMLET, son to the late, and nephew to the present king.</PERSONA>
<PERSONA>POLONIUS, lord chamberlain. </PERSONA>
<PERSONA>HORATIO, friend to Hamlet.</PERSONA>
<PERSONA>LAERTES, son to Polonius.</PERSONA>
<PERSONA>LUCIANUS, nephew to the king.</PERSONA>

<PGROUP>
<PERSONA>VOLTIMAND</PERSONA>
<PERSONA>CORNELIUS</PERSONA>
<PERSONA>ROSENCRANTZ</PERSONA>
<PERSONA>GUILDENSTERN</PERSONA>
<PERSONA>OSRIC</PERSONA>
<GRPDESCR>courtiers.</GRPDESCR>
</PGROUP>

<PERSONA>A Gentleman</PERSON

In [5]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub('<[^<]+>', "",text)
    text = re.sub('<!--',"",text)
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [69]:
hamlet_text = gutenberg.raw('blake-poems.txt')
macbeth_text = gutenberg.raw('shakespeare-caesar.txt')

In [70]:
hamlet_text = text_cleaner(hamlet_text[:int(len(hamlet_text)/10)])
macbeth_text= text_cleaner(macbeth_text[:int(len(macbeth_text)/10)])

In [66]:
print(macbeth_text)

Actus Primus. Scoena Prima. Enter Flauius, Murellus, and certaine Commoners ouer the Stage. Flauius. Hence: home you idle Creatures, get you home: Is this a Holiday? What, know you not (Being Mechanicall) you ought not walke Vpon a labouring day, without the signe Of your Profession? Speake, what Trade art thou? Car. Why Sir, a Carpenter Mur. Where is thy Leather Apron, and thy Rule? What dost thou with thy best Apparrell on? You sir, what Trade are you? Cobl. Truely Sir, in respect of a fine Workman, I am but as you would say, a Cobler Mur. But what Trade art thou? Answer me directly Cob. A Trade Sir, that I hope I may vse, with a safe Conscience, which is indeed Sir, a Mender of bad soules Fla. What Trade thou knaue? Thou naughty knaue, what Trade? Cobl. Nay I beseech you Sir, be not out with me: yet if you be out Sir, I can mend you Mur. What mean'st thou by that? Mend mee, thou sawcy Fellow? Cob. Why sir, Cobble you Fla. Thou art a Cobler, art thou? Cob. Truly sir, all that I liue 

In [9]:
nlp = spacy.load('en_core_web_sm')

In [19]:
shake_ham_doc = nlp(hamlet_text)

In [20]:
shake_mac_doc = nlp(macbeth_text)

In [21]:
hamlet_sents = [[sent, "Hamlet"] for sent in shake_ham_doc.sents]
macbeth_sents = [[sent,"Macbeth"] for sent in shake_mac_doc.sents]

In [22]:
print(len(macbeth_sents))

222


In [23]:
print(len(hamlet_sents))

340


In [24]:
print(hamlet_sents)

[[The Tragedy of Hamlet, Prince of Denmark Text placed in the public domain by Moby Lexical Tools, 1992., 'Hamlet'], [XML markup by Jon Bosak, 1996-1998., 'Hamlet'], [XML Styling done by Ajay Juneja, 1999., 'Hamlet'], [This work may be freely copied and distributed worldwide., 'Hamlet'], [Dramatis Personae CLAUDIUS, king of Denmark., 'Hamlet'], [HAMLET, son to the late, and nephew to the present king., 'Hamlet'], [POLONIUS, lord chamberlain., 'Hamlet'], [HORATIO, friend to Hamlet., 'Hamlet'], [LAERTES, son to Polonius., 'Hamlet'], [LUCIANUS, nephew to the king., 'Hamlet'], [VOLTIMAND CORNELIUS ROSENCRANTZ GUILDENSTERN OSRIC courtiers., 'Hamlet'], [A Gentleman A Priest., 'Hamlet'], [MARCELLUS BERNARDO officers., 'Hamlet'], [FRANCISCO, a soldier., 'Hamlet'], [REYNALDO, servant to Polonius., 'Hamlet'], [Players., 'Hamlet'], [Two Clowns, grave-diggers., 'Hamlet'], [FORTINBRAS, prince of Norway., 'Hamlet'], [A Captain., 'Hamlet'], [English Ambassadors., 'Hamlet'], [GERTRUDE, queen of Denmar

In [25]:
sentences = pd.DataFrame(hamlet_sents + macbeth_sents)
#sentences = sentences.drop([0,1,2,3])
sentences.head()

,0,1
0,"(The, Tragedy, of, Hamlet, ,, Prince, of, Denm...",Hamlet
1,"(XML, markup, by, Jon, Bosak, ,, 1996, -, 1998...",Hamlet
2,"(XML, Styling, done, by, Ajay, Juneja, ,, 1999...",Hamlet
3,"(This, work, may, be, freely, copied, and, dis...",Hamlet
4,"(Dramatis, Personae, CLAUDIUS, ,, king, of, De...",Hamlet


In [26]:
sentences[sentences[1]=='Macbeth']
#sentences = sentences.drop([3436,3437,3438,3439])
sentences[sentences[1]=='Macbeth']

,0,1
340,"(The, Tragedy, of, Macbeth, Text, placed, in, ...",Macbeth
341,"(XML, markup, by, Jon, Bosak, ,, 1996, -, 1998...",Macbeth
342,"(XML, Styling, done, by, Ajay, Juneja, ,, 1999...",Macbeth
343,"(This, work, may, be, freely, copied, and, dis...",Macbeth
344,"(Dramatis, Personae, DUNCAN, ,, king, of, Scot...",Macbeth
345,"(MALCOLM, DONALBAIN, his, sons, .)",Macbeth
346,"(MACBETH, BANQUO, generals, of, the, king, 's,...",Macbeth
347,"(MACDUFF, LENNOX, ROSS, MENTEITH, ANGUS, CAITH...",Macbeth
348,"(FLEANCE, ,, son, to, Banquo, .)",Macbeth
349,"(SIWARD, ,, Earl, of, Northumberland, ,, gener...",Macbeth


In [27]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source']   = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
macbethwords = bag_of_words(shake_ham_doc)
hamletwords  = bag_of_words(shake_mac_doc)

# Combine bags to create a set of unique words.
common_words = set(macbethwords + hamletwords)

In [28]:
macbethwords = bag_of_words(shake_ham_doc)
hamletwords = bag_of_words(shake_mac_doc)
common_words = set(macbethwords + hamletwords)

In [29]:
# Create our data frame with features. This can take a while to run.
sentences = pd.DataFrame(hamlet_sents + macbeth_sents)
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550


,FRANCISCO,delight,sick,reason,Cawdor,disjoint,witness,romage,Hath,thyself,...,house,nay,MENTEITH,week,rebel,Laertes,forfeit,impress,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, Tragedy, of, Hamlet, ,, Prince, of, Denm...",Hamlet
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(XML, markup, by, Jon, Bosak, ,, 1996, -, 1998...",Hamlet
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(XML, Styling, done, by, Ajay, Juneja, ,, 1999...",Hamlet
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(This, work, may, be, freely, copied, and, dis...",Hamlet
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Dramatis, Personae, CLAUDIUS, ,, king, of, De...",Hamlet


# BOW Implementation

In [31]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9821958456973294

Test set score: 0.6977777777777778


# TD-IDF Implementation

In [35]:
dir(shakespeare)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_encoding',
 '_fileids',
 '_get_root',
 '_root',
 '_tagset',
 '_unload',
 '_wrap_etree',
 'abspath',
 'abspaths',
 'citation',
 'encoding',
 'ensure_loaded',
 'fileids',
 'license',
 'open',
 'raw',
 'readme',
 'root',
 'unicode_repr',
 'words',
 'xml']

In [75]:
shake_text = macbeth_text+hamlet_text

In [76]:
import nltk
from nltk.corpus import gutenberg
nltk.download('punkt')
nltk.download('gutenberg')
import re
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\clyde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\clyde\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [77]:
blake=gutenberg.paras('blake-poems.txt')
shake=gutenberg.paras('shakespeare-caesar.txt')
info = blake+shake
#processing
info_paras=[]
for paragraph in info:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    info_paras.append(' '.join(para))

print(info_paras[0:4])

['[ Poems by William Blake 1789 ]', 'SONGS OF INNOCENCE AND OF EXPERIENCE and THE BOOK of THEL', 'SONGS OF INNOCENCE', 'INTRODUCTION']


In [78]:
info_paras[3]

'INTRODUCTION'

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test = train_test_split(info_paras, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=False, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
info_paras_tfidf=vectorizer.fit_transform(info_paras)
print("Number of features: %d" % info_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(info_paras_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_train[3])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 616
Original sentence: A DIVINE IMAGE
Tf_idf vector: {'work': 0.4306810971993211, 'dark': 0.38702254842340317, 'rose': 0.39850624886024244, 'bags': 0.4306810971993211, 'Tom': 0.39850624886024244, 'And': 0.4020281971216814}


# Adding parts of speech for BoW 

In [81]:
def pos_df(sentences):
    pos = []
    for sent in sentences[0]:
        for token in sent:
            pos = pos + [token.pos_]
    pos = set(pos)
    pos_init = np.zeros(shape=(len(pos),len(sentences)))
    df = pd.DataFrame(data=pos_init.T,index=np.arange(0,len(sentences)),columns=pos)
    
    for sent,ind in zip(sentences[0],np.arange(1,len(sentences))):    
        for token in sent:
            df.loc[ind,token.pos_] += 1 
    return df

In [89]:
df = pos_df(sentences)

In [90]:
X = word_counts.drop(['text_sentence','text_source'],1)
X = pd.concat([X,df],1)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.3,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9720101781170484

Test set score: 0.7337278106508875


In [103]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, X_test, y_test, cv=10)

In [104]:
scores

array([0.82352941, 0.76470588, 0.82352941, 0.70588235, 0.52941176,
       0.52941176, 0.52941176, 0.52941176, 0.64705882, 0.5625    ])